In [8]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [9]:
# PENDLE_26Dec24 = "0xC71BA0E3C1FB9CBcB15fbC677e78C99aC1bc590B"
# PENDLE_26Jun25 = "0xc0e2b9ECABcA12D5024B2C11788B1cFaf972E5aa"
contract_address = '0xc0e2b9ECABcA12D5024B2C11788B1cFaf972E5aa'

In [10]:

# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine, text
from sqlmodel import Session
from core import constants
from core.db import engine
from datetime import datetime, timedelta

# Create a session
session = Session(engine)

# Define the SQL query to get initiated withdrawals without completions
query = text(
    """
    WITH latest_initiated_withdrawals AS (
        SELECT 
            id,
            from_address,
            to_address,
            tx_hash,
            timestamp,
            input,
            block_number,
            ROW_NUMBER() OVER (PARTITION BY from_address ORDER BY timestamp DESC) AS rn
        FROM public.onchain_transaction_history
        WHERE method_id IN (:withdraw_method_id_1, :withdraw_method_id_2)
        AND to_address = ANY(:vault_addresses)
        AND timestamp >= :start_ts
        AND timestamp <= :end_ts
    ),
    has_later_completion AS (
        SELECT DISTINCT 
            i.from_address,
            i.tx_hash
        FROM latest_initiated_withdrawals i
        WHERE i.rn = 1
        AND EXISTS (
            SELECT 1
            FROM public.onchain_transaction_history c
            WHERE c.from_address = i.from_address
            AND to_address = ANY(:vault_addresses)
            AND c.method_id = :complete_method_id
            AND c.timestamp > i.timestamp
        )
    )
    SELECT *
    FROM latest_initiated_withdrawals i
    WHERE i.rn = 1
    AND NOT EXISTS (
        SELECT 1 
        FROM has_later_completion h 
        WHERE h.from_address = i.from_address
    );
    """
)

# Define parameters for the query
params = {
    "withdraw_method_id_1": "0xb51d1d4f",  # First initiate withdrawal method ID
    "withdraw_method_id_2": "0x087fad4c",  # Second initiate withdrawal method ID
    "complete_method_id": "0xe03ff7cb",     # Complete withdrawal method ID
    "vault_addresses": [contract_address.lower()],
    "start_ts": int((datetime.now() - timedelta(days=7)).timestamp()),  # Start timestamp is last 7 days
    "end_ts": int(datetime.now().timestamp())  # End timestamp is today
}

# Execute the query and fetch results
with session.begin():
    init_withdraws = session.execute(query, params).all()

# print(init_withdraws)
# Create a DataFrame from the results
# Create a DataFrame from the results
df_withdrawals = pd.DataFrame(init_withdraws, columns=["id", "from_address", "to_address", "tx_hash", "timestamp", "input", "block_number", "rn"])

# Close the database session
session.close()


# Display the DataFrame
df_withdrawals.head()  # Show the first few rows of the DataFrame

,id,from_address,to_address,tx_hash,timestamp,input,block_number,rn
0,78b5e9d2-4cd0-4147-a15d-433fcc17d4a0,0xc3c1223b8b87878731c4d6488984d89e64a09eff,0xc0e2b9ecabca12d5024b2c11788b1cfaf972e5aa,0xad87788090aa04ceeca4eba1479f5e7c393cb1cec99e...,1735624998,0x087fad4c,290430116,1


In [11]:
import json
from web3 import Web3
import pandas as pd
from core.config import settings
from core.abi_reader import read_abi


# Initialize Web3
# Replace 'YOUR_INFURA_URL' with your actual Infura or other Ethereum provider URL
w3 = Web3(Web3.HTTPProvider(settings.ARBITRUM_MAINNET_INFURA_URL))


# Load the ABI from the JSON file
with open('../config/pendlehedging_abi.json') as f:
    abi = json.load(f)

# Create a contract instance

pendle_vault = w3.eth.contract(address=contract_address, abi=abi)


In [12]:
# Function to get user withdrawal details
def get_user_withdrawals(user_address):
    try:
        # Call the getUserWithdraw function
        result = pendle_vault.functions.getUserWithdraw().call({'from': Web3.to_checksum_address(user_address)})
        
        # Extract ptWithdrawAmount and scWithdrawAmount
        withdrawalShares = result[0]
        ptWithdrawAmount = result[2]  # Assuming ptWithdrawAmount is the third element
        scWithdrawAmount = result[3]  # Assuming scWithdrawAmount is the fourth element
        
        # Convert to float
        ptWithdrawAmount_float = ptWithdrawAmount / 1e18
        scWithdrawAmount_float = scWithdrawAmount / 1e6
        withdrawalShares_float = withdrawalShares / 1e6
        
        return ptWithdrawAmount_float, scWithdrawAmount_float, withdrawalShares_float
    except Exception as e:
        print(f"Error fetching withdrawal details for {user_address}: {e}")
        return None, None

# Function to get the balance of a user
def get_user_balance(user_address):
    try:
        # Call the balanceOf function
        balance = pendle_vault.functions.balanceOf(Web3.to_checksum_address(user_address)).call()
        return balance / 1e18  # Convert to float if necessary
    except Exception as e:
        print(f"Error fetching balance for {user_address}: {e}")
        return None

# Iterate over each user in the DataFrame and get their withdrawal details
withdrawal_details = []
for index, row in df_withdrawals.iterrows():
    user_address = row['from_address']
    pt_amount, sc_amount, shares = get_user_withdrawals(user_address)
    # balance = get_user_balance(user_address)  # Get user balance
    withdrawal_details.append({
        "from_address": user_address,
        "ptWithdrawAmount": pt_amount,
        "scWithdrawAmount": sc_amount,
        "shares": shares,
    })

# Create a DataFrame for the withdrawal details
df_withdrawal_details = pd.DataFrame(withdrawal_details)

# Display the withdrawal details DataFrame
df_withdrawal_details

,from_address,ptWithdrawAmount,scWithdrawAmount,shares
0,0xc3c1223b8b87878731c4d6488984d89e64a09eff,0.06,200.0096,399.981214


In [13]:
# Function to get withdraw pool amounts
def get_withdraw_pool_amount():
    try:
        # Call the getWithdrawPoolAmount function
        result = pendle_vault.functions.getWithdrawPoolAmount().call()
        
        # Extract scWithdrawPoolAmount and ptWithdrawPoolAmount
        scWithdrawPoolAmount = result[0] / 1e6  # Convert to float
        ptWithdrawPoolAmount = result[1] / 1e18  # Convert to float
        
        return scWithdrawPoolAmount, ptWithdrawPoolAmount
    except Exception as e:
        print(f"Error fetching withdraw pool amounts: {e}")
        return None, None

# Get the withdraw pool amounts
sc_withdraw_pool_amount, pt_withdraw_pool_amount = get_withdraw_pool_amount()

# Calculate total Pendle withdrawal from df_withdrawal_details
total_sc_withdrawn = df_withdrawal_details['scWithdrawAmount'].sum() if 'scWithdrawAmount' in df_withdrawal_details else 0
total_pt_withdrawn = df_withdrawal_details['ptWithdrawAmount'].sum() if 'ptWithdrawAmount' in df_withdrawal_details else 0
total_shares_withdrawn = df_withdrawal_details['shares'].sum() if 'shares' in df_withdrawal_details else 0

# Create a report
report = {
    "Total SC Withdrawn": total_sc_withdrawn,
    "Total PT Withdrawn": total_pt_withdrawn,
    "Total Shares Withdrawn": total_shares_withdrawn,
    "Withdraw Pool SC Amount": sc_withdraw_pool_amount,
    "Withdraw Pool PT Amount": pt_withdraw_pool_amount,
    "============": "============",
    "Total SC Needed to Withdraw": total_sc_withdrawn - sc_withdraw_pool_amount,
    "Total PT Needed to Withdraw": total_pt_withdrawn - pt_withdraw_pool_amount
}

# Display the report
print("Withdrawal Report:")
for key, value in report.items():
    print(key, value)

if total_sc_withdrawn > sc_withdraw_pool_amount:
    print('DONT NEED TO WITHDRAW')


Withdrawal Report:
Total SC Withdrawn 200.0096
Total PT Withdrawn 0.06
Total Shares Withdrawn 399.981214
Withdraw Pool SC Amount 198.917405
Withdraw Pool PT Amount 0.06
============ ============
Total SC Needed to Withdraw 1.0921950000000038
Total PT Needed to Withdraw 0.0
DONT NEED TO WITHDRAW


In [14]:

import json

# Prepare the data for the JSON output
json_output = {
    "action": "retry_withdrawal_event",
    "details": {
        "pt_amount": round(report["Total PT Needed to Withdraw"], 2),
        "eth_amount": round(report["Total PT Needed to Withdraw"], 2),
        "amount_in_usd": round(report["Total SC Needed to Withdraw"], 2),
        "shares": 3404275515/1e6
    }
}

# Print the JSON text
print(json.dumps(json_output, indent=4))


{
    "action": "retry_withdrawal_event",
    "details": {
        "pt_amount": 0.0,
        "eth_amount": 0.0,
        "amount_in_usd": 1.09,
        "shares": 3404.275515
    }
}
